# GB Group cost of capital

In [1]:
import pandas as pd
import pandas_datareader as web
import numpy as np
import datetime as dt
import statsmodels.api as sm

## Cost of Equity

In [2]:
start_date = dt.datetime(2010, 12, 31)
end_date = dt.datetime(2021, 1, 1)

tickers = 'GBG.L'

In [3]:
# 10 years adjusted close price of GB Group and FTSE100
df = pd.DataFrame()

df[tickers] = web.DataReader(tickers, 'yahoo', start_date, end_date)['Close']

In [4]:
# 10 years adjusted close for market portfolio in the UK
df2 = pd.read_csv('HistoricalPrices.csv')
df2['Date'] = pd.to_datetime(df2['Date'], infer_datetime_format=True)
df2.set_index('Date', inplace=True)

In [5]:
df3 = pd.concat([df, df2['Close']], axis=1)
df3.dropna(inplace=True)
df = df3.rename(columns={'Close': 'FTSE100'})
df.head()

,GBG.L,FTSE100
Date,,
2010-12-31,34.25,5899.94
2011-01-04,34.50,6013.87
2011-01-05,34.75,6043.86
2011-01-06,34.75,6019.51
2011-01-07,34.75,5984.33


In [6]:
returns = df.copy()
for x in returns.columns:
    returns[x] = np.log(returns[x]) - np.log(returns[x].shift(1))

returns

,GBG.L,FTSE100
Date,,
2010-12-31,NaN,NaN
2011-01-04,0.007273,0.019126
2011-01-05,0.007220,0.004974
2011-01-06,0.000000,-0.004037
2011-01-07,0.000000,-0.005861
...,...,...
2020-12-23,0.002153,0.006578
2020-12-24,-0.003231,0.000979
2020-12-29,0.028710,0.015344


In [7]:
risk_free = 0.0163
returns['MRP'] = returns['FTSE100'] - risk_free
returns.dropna(inplace=True)
returns

,GBG.L,FTSE100,MRP
Date,,,
2011-01-04,0.007273,0.019126,0.002826
2011-01-05,0.007220,0.004974,-0.011326
2011-01-06,0.000000,-0.004037,-0.020337
2011-01-07,0.000000,-0.005861,-0.022161
2011-01-10,0.000000,-0.004695,-0.020995
...,...,...,...
2020-12-23,0.002153,0.006578,-0.009722
2020-12-24,-0.003231,0.000979,-0.015321
2020-12-29,0.028710,0.015344,-0.000956


In [8]:
X = sm.add_constant(returns['MRP'])    # add constant to the x vairable
y = returns['GBG.L']

model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  GBG.L   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     157.1
Date:                Tue, 16 Mar 2021   Prob (F-statistic):           5.37e-35
Time:                        23:47:46   Log-Likelihood:                 6354.5
No. Observations:                2521   AIC:                        -1.270e+04
Df Residuals:                    2519   BIC:                        -1.269e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0089      0.001     12.372      0.000       0.007       0.010
MRP            0.4644      0.037     12.532      0.000       0.392       0.537
==============================================================================
Omnibus:                      624.634   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            21609.791
Skew:                           0.462   Prob(JB):                         0.00
Kurtosis:                      17.313   Cond. No.                         95.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
beta = results.params['MRP']
beta

0.4644054150405105